In [7]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np

DEVICE = "mps"

In [11]:
a = [np.ones((16,16,3))]
np.stack(a).shape

(1, 16, 16, 3)

# Pose 2D Model

In [3]:
from simple_pose2d_model import get_pose2d_model

pose2d_model_config, pose2d_model = get_pose2d_model(
    pose2d_model_config_path="configs/halpe_26/resnet/256x192_res50_lr1e-3_1x.yaml",
    pose2d_checkpoint_path="pretrained_models/halpe26_fast_res50_256x192.pth",
    device=DEVICE
)

/Users/uinone/Desktop/AlphaPose/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/uinone/Desktop/AlphaPose/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/uinone/Desktop/AlphaPose/simple_pose2d_model.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://git

2D Pose model ready!


# Detection Model

In [4]:
from simple_detection_loader import get_detection_model

detection_model = get_detection_model(
    device=DEVICE, 
    pose2d_model_config=pose2d_model_config
)

# Pose 3D Model

In [5]:
from simple_pose3d_model import get_pose3d_model

pose3d_model = get_pose3d_model(
    pose3d_config_path="MotionBERT/configs/pose3d/MB_ft_h36m.yaml",
    pose3d_weight_path="MotionBERT/weights/MB_ft_h36m.bin",
    device=DEVICE
)

/Users/uinone/Desktop/AlphaPose/simple_pose3d_model.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_ = torch.load(pose3d_weight_path, map_location=lambda sto

3D pose model ready!


# DataWriter

In [6]:
from simpe_datawriter import DataWriter
from simple_detection_loader import DetectionOpt

In [7]:
pose2d_estimator = DataWriter(
    pose2d_model_config, 
    DetectionOpt(DEVICE)
)

# Visualize

In [ ]:
import time
def vid_frame():
    cap = cv2.VideoCapture("examples/demo/custom_example_6.mp4")
    while True:
        ret, img = cap.read()

        if not ret:
            break
            
        yield img

In [2]:
import time
def webcam_recorder(sec: float = 5, save_path:str = None):
    cam = cv2.VideoCapture(0)

    frames = []

    if save_path is not None:
        frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (frame_width, frame_height))

        start = time.time()
        end = time.time()
        while (end - start) < sec:
            ret, img = cam.read()
            
            frames.append(img)
            out.write(img)

            if not ret:
                break

            end = time.time()

        out.release()
    else:
        print("FPS :", cam.get(cv2.CAP_PROP_FPS))

        start = time.time()
        end = time.time()
        while (end - start) < sec:
            ret, img = cam.read()

            frames.append(img)

            if not ret:
                break

            end = time.time()

    cam.release()
    
    return frames

In [3]:
frames = webcam_recorder(sec=5)

2024-11-12 14:07:31.519 python[16062:165259] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


FPS : 15.0


In [6]:
np.stack(frames).shape

(151, 1080, 1920, 3)

In [9]:
vf = vid_frame()
pose2d_inputs = []

with torch.no_grad():
    while True:
        try:
            img = next(vf)

            img_h, img_w = img.shape[:2]
            # inps, orig_img, boxes, scores, ids, cropped_boxes
            detection_outp = detection_model.process(img).read()

            (inps, orig_img, boxes, scores, ids, cropped_boxes) = detection_outp
            pose2d_input = (inps[0:1], orig_img, boxes[0:1], scores[0:1], ids[0:1], cropped_boxes[0:1])


            # box_img = img.copy()
            # for box in boxes:
            #     l,t,r,b = np.array(box, np.int32)
            #     cv2.rectangle(box_img, (l,t), (r,b), (0,0,255), 3)
            #     break

            # plt.imshow(box_img)
            pose2d_inputs.append(pose2d_input)
            print(len(pose2d_inputs), end=" ")
        except Exception as e:
            print("Done")
            break

Loading YOLO model..
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272

# Pose 2D Visualize

In [10]:
pose3d_inputs = []

with torch.no_grad():
    for detection_outp in pose2d_inputs:
        (inps, orig_img, boxes, scores, ids, cropped_boxes) = detection_outp
        hm = pose2d_model(inps.to("mps")).cpu()
        pose2d_estimator.save(boxes, scores, ids, hm, cropped_boxes, orig_img)
        hp_outp = pose2d_estimator.start()
        pose3d_inputs.append(hp_outp)

# from simple_pose2d_visualizer import get_pose2d_result
# def _a():
#     with torch.no_grad():
#         for detection_outp in pose2d_inputs:
#             (inps, orig_img, boxes, scores, ids, cropped_boxes) = detection_outp
#             hm = pose2d_model(inps.to("mps")).cpu()
#             pose2d_estimator.save(boxes, scores, ids, hm, cropped_boxes, orig_img)
#             hp_outp = pose2d_estimator.start()

#             plt.imshow(get_pose2d_result(orig_img[:,:,::-1], hp_outp))
#             yield

# a = _a()

# 3D Pose estimation

In [11]:
keypoints = [torch.concat([hp_outp["result"][0]['keypoints'], hp_outp["result"][0]['kp_score']], dim=-1) for hp_outp in pose3d_inputs]
keypoints = torch.stack(keypoints, dim=0)
keypoints.shape

torch.Size([351, 26, 3])

In [12]:
from MotionBERT.lib.data.dataset_wild import WildDetDataset

wild_dataset = WildDetDataset(
    clip_len=keypoints.shape[0],
    image_size=pose2d_inputs[0][1].shape[:2][::-1],
)

for keypoint in keypoints:
    wild_dataset.add_data(torch.unsqueeze(keypoint, dim=0))

keypoints2d = np.concatenate(wild_dataset.frames, axis=0)
keypoints_scores = keypoints2d[...,2]
keypoints2d = torch.FloatTensor(keypoints2d)
keypoints2d = torch.unsqueeze(keypoints2d, dim=0)
keypoints2d.shape

torch.Size([1, 351, 17, 3])

In [14]:
import MotionBERT.lib.utils.vismo as vismo
from MotionBERT.lib.utils.vismo import motion2video_3d, pixel2world_vis_motion
import importlib

importlib.reload(vismo)

with torch.no_grad():
    pose3d_outp = pose3d_model(keypoints2d[:,keypoints2d.shape[1]-243:].to(DEVICE)).cpu()[0]


motion = np.transpose(pose3d_outp, (1,2,0))   #(1,17,D) -> (17,D,1) 

motion_world = pixel2world_vis_motion(motion, dim=3)

In [15]:
orig_imges = []
for detection_output in pose2d_inputs[keypoints2d.shape[1]-243:]:
    orig_img = detection_output[1]
    l,t,r,b = np.array(detection_output[2][0], np.int32)

    orig_imges.append(orig_img[t:b,l:r])
print(len(orig_imges))

243


/var/folders/xc/8jm9lm450w78w8ccr693xgz80000gn/T/ipykernel_11540/3185775907.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  l,t,r,b = np.array(detection_output[2][0], np.int32)


In [17]:
motion2video_3d(
    orig_imges,
    motion_world,
    keypoints_scores, 
    './custom_example_out_6.mp4', 
    elivation=80,
    angle=0,
    fps=30
)

100%|██████████| 243/243 [00:24<00:00,  9.88it/s]
